# 🚀 AI Career Twin
## Interactive Digital Clone for Skill Gap Detection & Career Simulation

---

```
  ╔══════════════════════════════════════════════════════════════╗
  ║   YOUR PERSONAL AI CAREER INTELLIGENCE SYSTEM               ║
  ║   ─────────────────────────────────────────────────────     ║
  ║   📄 Resume Analysis    →  Extracts your skill DNA         ║
  ║   🎯 Gap Detection      →  Finds what's missing             ║
  ║   🎤 Interview Sim      →  Prepares you for the room        ║
  ║   🗺️  Career Roadmap    →  Plots your growth path           ║
  ║   📈 Trajectory Chart   →  Visualizes your future           ║
  ╚══════════════════════════════════════════════════════════════╝
```

### ⚡ Powered by Google Gemini AI

**Run cells top to bottom. Each cell builds on the last.**

---

## 📦 Cell 1 — Install & Import

In [ ]:
# ─── Install dependencies ────────────────────────────────────────────────────
import subprocess, sys

packages = [
    'google-generativeai',
    'PyPDF2',
    'python-docx',
    'plotly',
    'pandas',
    'matplotlib',
    'kaleido',
]

print('⏳ Installing packages...')
for pkg in packages:
    result = subprocess.run(
        [sys.executable, '-m', 'pip', 'install', pkg, '-q'],
        capture_output=True, text=True
    )
    status = '✅' if result.returncode == 0 else '❌'
    print(f'   {status} {pkg}')

print('\n⏳ Importing libraries...')
import os, re, json, time, textwrap, base64, io
from pathlib import Path
from typing import List, Dict, Optional, Tuple
from dataclasses import dataclass, field
from datetime import datetime
from collections import defaultdict

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.patches import FancyBboxPatch
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

import google.generativeai as genai

try:
    import PyPDF2
    PDF_AVAILABLE = True
except ImportError:
    PDF_AVAILABLE = False

try:
    import docx
    DOCX_AVAILABLE = True
except ImportError:
    DOCX_AVAILABLE = False

# Google Colab utilities
try:
    from google.colab import files as colab_files
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

from IPython.display import display, HTML, Markdown, clear_output
import ipywidgets as widgets

print('\n✅ All imports successful!')
print(f'   Running in Colab : {IN_COLAB}')
print(f'   PDF support      : {PDF_AVAILABLE}')
print(f'   DOCX support     : {DOCX_AVAILABLE}')

## 🔑 Cell 2 — Configure Gemini API Key

In [ ]:
# ─── API Key Configuration ───────────────────────────────────────────────────
# Get your FREE Gemini API key at: https://aistudio.google.com/app/apikey

display(HTML('''
<div style="background:linear-gradient(135deg,#1a1a2e,#16213e);border:1px solid #0f3460;
            border-radius:12px;padding:20px;color:#e0e0e0;font-family:monospace;">
  <h3 style="color:#00d4ff;margin-top:0">🔑 API Key Required</h3>
  <p style="margin:5px 0">Get your <b>FREE</b> Gemini API key from:</p>
  <a href="https://aistudio.google.com/app/apikey" target="_blank"
     style="color:#ff6b6b;font-size:1.1em">👉 https://aistudio.google.com/app/apikey</a>
  <p style="margin-top:10px;color:#aaa;font-size:0.85em">
    Free tier: 15 requests/min · 1M tokens/min · No credit card needed</p>
</div>
'''))

API_KEY = ""  # ← PASTE YOUR KEY HERE

# ── Auto-detect from Colab secrets ──
if not API_KEY and IN_COLAB:
    try:
        from google.colab import userdata
        API_KEY = userdata.get('GEMINI_API_KEY')
        if API_KEY:
            print('✅ API key loaded from Colab Secrets')
    except Exception:
        pass

# ── Auto-detect from environment ──
if not API_KEY:
    API_KEY = os.environ.get('GEMINI_API_KEY', '')
    if API_KEY:
        print('✅ API key loaded from environment variable')

# ── Prompt if still missing ──
if not API_KEY:
    import getpass
    print('\nNo API key found automatically.')
    API_KEY = getpass.getpass('🔑 Enter your Gemini API key (input hidden): ').strip()

# ── Configure and validate ──
if not API_KEY:
    raise ValueError('❌ API key is required. Please get one from https://aistudio.google.com/app/apikey')

genai.configure(api_key=API_KEY)

# ── Test connection ──
try:
    model = genai.GenerativeModel('gemini-1.5-flash')
    test  = model.generate_content('Say exactly: CONNECTION_OK')
    if 'CONNECTION_OK' in test.text or test.text:
        print('✅ Gemini API connected successfully!')
        print(f'   Model: gemini-1.5-flash')
        print(f'   Key  : {API_KEY[:8]}...{API_KEY[-4:]}')
except Exception as e:
    raise ConnectionError(f'❌ API connection failed: {e}\nCheck your key and try again.')

## 🏗️ Cell 3 — Core Engine (Data Models + AI Brain)

In [ ]:
# ─── Data Models ─────────────────────────────────────────────────────────────

@dataclass
class SkillProfile:
    raw_skills: Dict[str, List[str]] = field(default_factory=dict)
    experience_years: float = 0.0
    current_role: str = ''
    education: str = ''
    certifications: List[str] = field(default_factory=list)
    projects: List[str] = field(default_factory=list)
    all_skills: List[str] = field(default_factory=list)

    def __post_init__(self):
        self.all_skills = sorted({s for v in self.raw_skills.values() for s in v})


@dataclass
class JobProfile:
    title: str = ''
    company: str = ''
    required_skills: List[str] = field(default_factory=list)
    nice_to_have: List[str] = field(default_factory=list)
    experience_required: float = 0.0
    seniority: str = ''
    domain: str = ''


@dataclass
class GapAnalysis:
    critical_gaps: List[str] = field(default_factory=list)
    nice_to_have_gaps: List[str] = field(default_factory=list)
    matching_skills: List[str] = field(default_factory=list)
    match_score: float = 0.0
    readiness_level: str = ''
    gap_priority: Dict[str, int] = field(default_factory=dict)   # skill → weeks to learn


@dataclass
class CareerTwin:
    resume_text: str = ''
    jd_text: str = ''
    skill_profile: Optional[SkillProfile] = None
    job_profile: Optional[JobProfile] = None
    gap_analysis: Optional[GapAnalysis] = None
    roadmap: List[Dict] = field(default_factory=list)
    interview_qa: List[Dict] = field(default_factory=list)
    trajectory: List[Dict] = field(default_factory=list)


# ─── Skill Knowledge Base ─────────────────────────────────────────────────────
SKILL_DB = {
    'languages': [
        'python','java','javascript','typescript','c++','c#','go','rust','swift',
        'kotlin','r','scala','matlab','sql','bash','perl','ruby','php','dart','julia'
    ],
    'frameworks': [
        'react','angular','vue','django','flask','fastapi','spring','express',
        'tensorflow','pytorch','keras','scikit-learn','pandas','numpy','spark',
        'hadoop','airflow','mlflow','huggingface','langchain','nextjs','svelte'
    ],
    'cloud_devops': [
        'aws','azure','gcp','docker','kubernetes','terraform','ansible','jenkins',
        'github actions','ci/cd','helm','istio','prometheus','grafana','datadog'
    ],
    'databases': [
        'postgresql','mysql','mongodb','redis','elasticsearch','cassandra',
        'dynamodb','bigquery','snowflake','neo4j','pinecone','weaviate'
    ],
    'concepts': [
        'machine learning','deep learning','nlp','computer vision','data science',
        'devops','agile','scrum','microservices','rest api','graphql','grpc',
        'system design','distributed systems','llm','rag','fine-tuning','mlops',
        'data engineering','feature engineering','a/b testing','statistics'
    ],
    'soft_skills': [
        'leadership','communication','problem solving','teamwork','mentoring',
        'project management','stakeholder management','technical writing'
    ]
}

LEARNING_TIME = {
    # skill → (weeks_for_beginner, weeks_for_experienced)
    'kubernetes': (8, 4), 'terraform': (6, 3), 'spark': (10, 5),
    'rust': (16, 8), 'go': (8, 4), 'typescript': (4, 2),
    'pytorch': (10, 5), 'mlflow': (3, 1), 'airflow': (4, 2),
    'graphql': (3, 1), 'redis': (3, 1), 'elasticsearch': (5, 3),
    'docker': (4, 2), 'aws': (12, 6), 'azure': (12, 6),
    'react': (8, 4), 'fastapi': (3, 1), 'langchain': (4, 2),
    'llm': (8, 4), 'rag': (6, 3), 'fine-tuning': (8, 4),
}


# ─── Document Parser ──────────────────────────────────────────────────────────
class DocumentParser:
    @staticmethod
    def from_pdf(data: bytes) -> str:
        if not PDF_AVAILABLE:
            raise ImportError('PyPDF2 not installed')
        reader = PyPDF2.PdfReader(io.BytesIO(data))
        return '\n'.join(page.extract_text() or '' for page in reader.pages)

    @staticmethod
    def from_docx(data: bytes) -> str:
        if not DOCX_AVAILABLE:
            raise ImportError('python-docx not installed')
        doc = docx.Document(io.BytesIO(data))
        return '\n'.join(p.text for p in doc.paragraphs if p.text.strip())

    @staticmethod
    def from_text(data: bytes) -> str:
        for enc in ('utf-8', 'latin-1', 'cp1252'):
            try:
                return data.decode(enc)
            except UnicodeDecodeError:
                continue
        return data.decode('utf-8', errors='replace')

    @classmethod
    def parse(cls, filename: str, data: bytes) -> str:
        ext = Path(filename).suffix.lower()
        if ext == '.pdf':   return cls.from_pdf(data)
        if ext == '.docx':  return cls.from_docx(data)
        return cls.from_text(data)


# ─── Local Skill Extractor ────────────────────────────────────────────────────
def extract_skills_local(text: str) -> Dict[str, List[str]]:
    t = text.lower()
    found = {}
    for category, skills in SKILL_DB.items():
        matched = [s for s in skills
                   if re.search(r'(?<![\w-])' + re.escape(s) + r'(?![\w-])', t)]
        if matched:
            found[category] = matched
    return found


def extract_experience_years(text: str) -> float:
    patterns = [
        r'(\d+)\+?\s*years?\s+(?:of\s+)?experience',
        r'experience\s*(?:of|:)?\s*(\d+)\+?\s*years?',
        r'(\d+)\+?\s*yrs?\s+experience',
    ]
    for p in patterns:
        m = re.search(p, text.lower())
        if m:
            return float(m.group(1))
    return 0.0


# ─── JSON Parser ─────────────────────────────────────────────────────────────
def safe_parse_json(text: str) -> dict:
    for attempt in [
        lambda t: json.loads(t),
        lambda t: json.loads(re.search(r'```(?:json)?\s*([\s\S]*?)```', t).group(1).strip()),
        lambda t: json.loads(re.search(r'\{[\s\S]*\}', t).group(0)),
    ]:
        try:
            return attempt(text)
        except Exception:
            continue
    return {}


print('✅ Data models defined')
print('✅ Skill database loaded  (%d categories, %d total skills)' % (
    len(SKILL_DB), sum(len(v) for v in SKILL_DB.values())))
print('✅ Document parser ready  (PDF / DOCX / TXT)')
print('✅ Core engine initialized')

## 🤖 Cell 4 — AI Analysis Engine

In [ ]:
# ─── Gemini AI Analysis Engine ────────────────────────────────────────────────

class CareerAI:
    def __init__(self, api_key: str):
        genai.configure(api_key=api_key)
        self.model   = genai.GenerativeModel('gemini-1.5-flash')
        self.flash   = genai.GenerativeModel('gemini-1.5-flash')
        self._delay  = 1.5   # rate limit buffer (sec)

    def _ask(self, prompt: str, json_mode: bool = False) -> str:
        time.sleep(self._delay)
        cfg = genai.types.GenerationConfig(
            temperature=0.3,
            max_output_tokens=4096,
        )
        resp = self.model.generate_content(prompt, generation_config=cfg)
        return resp.text

    # ── 1. Resume Deep Analysis ──────────────────────────────────────────────
    def analyze_resume(self, resume_text: str, local_skills: dict) -> SkillProfile:
        prompt = f"""You are an expert technical recruiter and career coach.
Analyze this resume and extract structured information.

RESUME:
{resume_text[:4000]}

LOCAL SKILL EXTRACTION (already found): {json.dumps(local_skills)}

Return ONLY a JSON object with these exact keys:
{{
  "current_role": "string - most recent job title",
  "experience_years": number,
  "education": "string - highest degree",
  "certifications": ["list", "of", "certifications"],
  "projects": ["list", "of", "notable", "projects"],
  "strengths": ["top", "3", "technical", "strengths"],
  "seniority_level": "Junior|Mid|Senior|Lead|Principal",
  "domain_expertise": "primary domain e.g. ML/Backend/Frontend/DevOps",
  "inferred_skills": ["skills implied but not explicitly stated"]
}}

Return ONLY the JSON. No explanation."""

        raw  = self._ask(prompt, json_mode=True)
        data = safe_parse_json(raw)

        # Merge AI inferred skills into local
        merged = dict(local_skills)
        inferred = data.get('inferred_skills', [])
        if inferred:
            merged['inferred'] = [s.lower() for s in inferred]

        return SkillProfile(
            raw_skills       = merged,
            experience_years = float(data.get('experience_years', 0)),
            current_role     = data.get('current_role', 'Not specified'),
            education        = data.get('education', 'Not specified'),
            certifications   = data.get('certifications', []),
            projects         = data.get('projects', []),
        )

    # ── 2. Job Description Parsing ───────────────────────────────────────────
    def parse_job_description(self, jd_text: str, local_skills: dict) -> JobProfile:
        prompt = f"""Parse this job description and extract requirements.

JOB DESCRIPTION:
{jd_text[:3000]}

Return ONLY a JSON object:
{{
  "title": "exact job title",
  "company": "company name or Unknown",
  "required_skills": ["list of must-have skills"],
  "nice_to_have": ["list of optional skills"],
  "experience_required": number,
  "seniority": "Junior|Mid|Senior|Lead|Staff|Principal",
  "domain": "primary domain e.g. ML Engineering/Backend/Data Science",
  "key_responsibilities": ["top 3 responsibilities"]
}}

Return ONLY the JSON. No explanation."""

        raw  = self._ask(prompt, json_mode=True)
        data = safe_parse_json(raw)

        req = [s.lower() for s in data.get('required_skills', [])]
        nth = [s.lower() for s in data.get('nice_to_have', [])]
        local_flat = [s for v in local_skills.values() for s in v]
        req  = list(set(req) | set(local_flat))

        return JobProfile(
            title               = data.get('title', 'Unknown Role'),
            company             = data.get('company', 'Unknown Company'),
            required_skills     = req,
            nice_to_have        = nth,
            experience_required = float(data.get('experience_required', 0)),
            seniority           = data.get('seniority', 'Mid'),
            domain              = data.get('domain', 'Tech'),
        )

    # ── 3. Gap Analysis ──────────────────────────────────────────────────────
    def analyze_gaps(self, profile: SkillProfile, job: JobProfile) -> GapAnalysis:
        resume_set = set(profile.all_skills)
        jd_set     = set(s.lower() for s in job.required_skills)
        nth_set    = set(s.lower() for s in job.nice_to_have)

        matching       = sorted(resume_set & jd_set)
        critical_gaps  = sorted(jd_set - resume_set)
        nth_gaps       = sorted((nth_set - resume_set) - jd_set)

        total    = max(len(jd_set), 1)
        score    = round(len(matching) / total * 100, 1)

        if   score >= 80: level = '🟢 Highly Qualified'
        elif score >= 60: level = '🟡 Strong Candidate'
        elif score >= 40: level = '🟠 Needs Preparation'
        else:             level = '🔴 Significant Gap'

        # Estimate learning time per gap
        exp = profile.experience_years
        gap_priority = {}
        for skill in critical_gaps:
            beg, senior = LEARNING_TIME.get(skill, (6, 3))
            weeks = senior if exp >= 3 else beg
            gap_priority[skill] = weeks

        return GapAnalysis(
            critical_gaps   = critical_gaps,
            nice_to_have_gaps = nth_gaps,
            matching_skills = matching,
            match_score     = score,
            readiness_level = level,
            gap_priority    = gap_priority,
        )

    # ── 4. Learning Roadmap ──────────────────────────────────────────────────
    def generate_roadmap(self, profile: SkillProfile, job: JobProfile, gaps: GapAnalysis) -> List[Dict]:
        if not gaps.critical_gaps:
            return [{'phase': 1, 'title': 'Polish & Apply',
                     'duration': '2 weeks',
                     'skills': list(gaps.matching_skills[:5]),
                     'actions': ['Update portfolio', 'Apply to roles', 'Network actively'],
                     'resources': ['LinkedIn', 'GitHub', 'Target company blogs']}]

        gap_str  = ', '.join(gaps.critical_gaps[:8])
        nth_str  = ', '.join(gaps.nice_to_have_gaps[:5])
        prompt = f"""You are a senior engineering career coach.
Create a personalized learning roadmap.

CANDIDATE: {profile.experience_years} years exp, current role: {profile.current_role}
TARGET ROLE: {job.title} at {job.company} ({job.seniority} level)
CRITICAL SKILL GAPS: {gap_str}
NICE-TO-HAVE GAPS: {nth_str}
CURRENT SKILLS: {', '.join(profile.all_skills[:15])}

Return ONLY a JSON array of phases (3-5 phases):
[
  {{
    "phase": 1,
    "title": "Phase title e.g. Foundation Sprint",
    "duration": "e.g. 3 weeks",
    "skills": ["skill1", "skill2"],
    "actions": ["specific action 1", "specific action 2", "specific action 3"],
    "resources": ["Free resource 1", "Platform/course 2"],
    "milestone": "what success looks like at end of phase"
  }}
]

Make it practical, specific, and achievable. Return ONLY the JSON array."""

        raw  = self._ask(prompt)
        data = safe_parse_json(raw)
        if isinstance(data, list) and data:
            return data
        # Fallback if JSON parsing fails
        phases = []
        sorted_gaps = sorted(gaps.gap_priority.items(), key=lambda x: x[1])
        chunk = max(1, len(sorted_gaps) // 3)
        for i, batch_start in enumerate(range(0, len(sorted_gaps), chunk)):
            batch = sorted_gaps[batch_start:batch_start + chunk]
            phases.append({
                'phase':     i + 1,
                'title':     f'Phase {i+1}: {"Foundation" if i==0 else "Build" if i==1 else "Mastery"}',
                'duration':  f'{sum(w for _, w in batch)} weeks',
                'skills':    [s for s, _ in batch],
                'actions':   [f'Study {s}' for s, _ in batch[:3]],
                'resources': ['Udemy', 'YouTube', 'Official Docs'],
                'milestone': f'Complete projects using {[s for s, _ in batch[:2]]}'
            })
        return phases

    # ── 5. Interview Simulator ───────────────────────────────────────────────
    def generate_interview_questions(self, profile: SkillProfile, job: JobProfile,
                                     gaps: GapAnalysis) -> List[Dict]:
        prompt = f"""You are a senior interviewer at a top tech company.
Generate realistic interview questions for this candidate.

CANDIDATE PROFILE:
- Experience: {profile.experience_years} years
- Current role: {profile.current_role}
- Skills: {', '.join(profile.all_skills[:15])}

TARGET ROLE: {job.title} ({job.seniority}) at {job.company}
DOMAIN: {job.domain}

Generate exactly 8 questions across these types. Return ONLY a JSON array:
[
  {{
    "type": "Technical|Behavioral|System Design|Situational",
    "difficulty": "Easy|Medium|Hard",
    "question": "the actual question text",
    "what_to_cover": ["key point 1", "key point 2", "key point 3"],
    "red_flags": "what a bad answer looks like",
    "sample_strong_answer": "2-3 sentence strong answer framework"
  }}
]

Mix: 3 Technical, 2 Behavioral, 2 System Design, 1 Situational.
Return ONLY the JSON array."""

        raw  = self._ask(prompt)
        data = safe_parse_json(raw)
        if isinstance(data, list) and data:
            return data
        # Fallback
        return [
            {'type': 'Technical', 'difficulty': 'Medium',
             'question': f'Walk me through your experience with {profile.all_skills[0] if profile.all_skills else "your primary skill"}.',
             'what_to_cover': ['Depth of knowledge', 'Real project examples', 'Trade-offs'],
             'red_flags': 'Vague answers without concrete examples',
             'sample_strong_answer': 'Use STAR method: Situation, Task, Action, Result'},
        ]

    # ── 6. Career Trajectory ─────────────────────────────────────────────────
    def predict_trajectory(self, profile: SkillProfile, job: JobProfile,
                            gaps: GapAnalysis) -> List[Dict]:
        prompt = f"""You are a career strategist. Predict this person's career trajectory.

CURRENT STATE:
- Role: {profile.current_role}
- Experience: {profile.experience_years} years
- Skills: {', '.join(profile.all_skills[:10])}
- Match score for target: {gaps.match_score}%

TARGET: {job.title} at {job.seniority} level in domain: {job.domain}

Predict career milestones for 5 years. Return ONLY a JSON array:
[
  {{
    "year": 0,
    "label": "Now",
    "role": "{profile.current_role}",
    "salary_range": "$X - $Y USD",
    "readiness_score": {gaps.match_score},
    "key_achievement": "current state description",
    "skills_to_add": []
  }},
  {{
    "year": 1,
    "label": "Year 1",
    "role": "next step role",
    "salary_range": "$X - $Y USD",
    "readiness_score": number between 0-100,
    "key_achievement": "what they should have achieved",
    "skills_to_add": ["skill1", "skill2"]
  }}
  // Continue for years 2, 3, 4, 5
]

Make salary ranges realistic for {job.domain} in USD. Return ONLY the JSON array."""

        raw  = self._ask(prompt)
        data = safe_parse_json(raw)
        if isinstance(data, list) and len(data) >= 3:
            return data
        # Fallback trajectory
        base_salary = 80000 if profile.experience_years < 3 else 120000
        return [
            {'year': i, 'label': f'Year {i}' if i > 0 else 'Now',
             'role': [profile.current_role, job.title, f'Senior {job.title}',
                      f'Lead {job.title}', 'Principal Engineer', 'Staff Engineer'][min(i, 5)],
             'salary_range': f'${base_salary + i*15000:,} - ${base_salary + i*25000:,}',
             'readiness_score': min(100, gaps.match_score + i * 15),
             'key_achievement': f'Year {i} milestone',
             'skills_to_add': gaps.critical_gaps[i*2:(i+1)*2] if i < 3 else []}
            for i in range(6)
        ]


# ── Instantiate engine ────────────────────────────────────────────────────────
ai_engine = CareerAI(api_key=API_KEY)
twin      = CareerTwin()

print('✅ CareerAI engine initialized')
print('✅ CareerTwin session started')
print('\nModules ready:')
print('  • analyze_resume()             — deep resume parsing')
print('  • parse_job_description()      — JD intelligence')
print('  • analyze_gaps()               — skill gap detection')
print('  • generate_roadmap()           — personalized learning plan')
print('  • generate_interview_questions()— interview simulation')
print('  • predict_trajectory()         — 5-year career forecast')

## 📊 Cell 5 — Visualization Engine

In [ ]:
# ─── Visualization Engine ─────────────────────────────────────────────────────

COLORS = {
    'primary'   : '#00d4ff',
    'accent'    : '#ff6b6b',
    'success'   : '#2ecc71',
    'warning'   : '#f39c12',
    'bg'        : '#0d1117',
    'bg2'       : '#161b22',
    'text'      : '#c9d1d9',
    'purple'    : '#a855f7',
    'orange'    : '#fb923c',
}


def display_header(title: str, subtitle: str = ''):
    html = f'''
    <div style="background:linear-gradient(135deg,#0d1117,#1a1a2e,#16213e);
                border:1px solid #30363d;border-radius:16px;
                padding:28px 32px;margin:16px 0;">
      <h1 style="color:#00d4ff;font-family:'Courier New',monospace;margin:0;
                 font-size:1.8em;letter-spacing:2px;">{title}</h1>
      {f'<p style="color:#8b949e;margin:8px 0 0;font-size:1em">{subtitle}</p>' if subtitle else ''}
    </div>'''
    display(HTML(html))


def display_score_card(score: float, level: str, matching: List[str], gaps: List[str]):
    color = ('#2ecc71' if score >= 80 else
             '#f39c12' if score >= 60 else
             '#e67e22' if score >= 40 else '#e74c3c')

    match_pills = ' '.join(
        f'<span style="background:#1e3a2e;color:#2ecc71;border:1px solid #2ecc71;'
        f'border-radius:20px;padding:3px 10px;font-size:0.78em;margin:2px;display:inline-block">{s}</span>'
        for s in matching[:12])

    gap_pills = ' '.join(
        f'<span style="background:#3a1e1e;color:#ff6b6b;border:1px solid #ff6b6b;'
        f'border-radius:20px;padding:3px 10px;font-size:0.78em;margin:2px;display:inline-block">{s}</span>'
        for s in gaps[:12])

    html = f'''
    <div style="background:#161b22;border:1px solid #30363d;border-radius:16px;
                padding:24px;margin:12px 0;">
      <div style="display:flex;align-items:center;gap:32px;flex-wrap:wrap">

        <!-- Score Circle -->
        <div style="text-align:center;min-width:120px">
          <div style="width:120px;height:120px;border-radius:50%;
                      border:6px solid {color};display:flex;align-items:center;
                      justify-content:center;margin:0 auto;
                      box-shadow:0 0 30px {color}44;">
            <div>
              <div style="color:{color};font-size:2em;font-weight:900;line-height:1">{score:.0f}%</div>
              <div style="color:#8b949e;font-size:0.65em">MATCH</div>
            </div>
          </div>
          <div style="color:{color};margin-top:8px;font-size:0.85em;font-weight:600">{level}</div>
        </div>

        <!-- Skills breakdown -->
        <div style="flex:1;min-width:200px">
          <div style="margin-bottom:12px">
            <div style="color:#2ecc71;font-size:0.8em;font-weight:700;letter-spacing:1px;
                        margin-bottom:6px">✅ MATCHING SKILLS ({len(matching)})</div>
            <div>{match_pills if match_pills else '<span style="color:#666">None detected</span>'}</div>
          </div>
          <div>
            <div style="color:#ff6b6b;font-size:0.8em;font-weight:700;letter-spacing:1px;
                        margin-bottom:6px">❌ SKILL GAPS ({len(gaps)})</div>
            <div>{gap_pills if gap_pills else '<span style="color:#2ecc71">No critical gaps! 🎉</span>'}</div>
          </div>
        </div>
      </div>
    </div>'''
    display(HTML(html))


def plot_skill_radar(profile: SkillProfile, job: JobProfile, gaps: GapAnalysis):
    categories = list(SKILL_DB.keys())
    resume_vals, jd_vals = [], []

    for cat in categories:
        cat_skills  = set(SKILL_DB[cat])
        res_match   = len(cat_skills & set(profile.all_skills))
        res_score   = min(100, res_match / max(1, len(cat_skills)) * 400)
        resume_vals.append(round(res_score, 1))

        jd_match    = len(cat_skills & set(job.required_skills))
        jd_score    = min(100, jd_match / max(1, len(cat_skills)) * 400)
        jd_vals.append(round(jd_score, 1))

    labels = [c.replace('_', ' ').title() for c in categories]

    fig = go.Figure()
    fig.add_trace(go.Scatterpolar(
        r=resume_vals + [resume_vals[0]], theta=labels + [labels[0]],
        fill='toself', name='Your Profile',
        line=dict(color='#00d4ff', width=2),
        fillcolor='rgba(0,212,255,0.15)'
    ))
    fig.add_trace(go.Scatterpolar(
        r=jd_vals + [jd_vals[0]], theta=labels + [labels[0]],
        fill='toself', name='Job Requirements',
        line=dict(color='#ff6b6b', width=2),
        fillcolor='rgba(255,107,107,0.15)'
    ))
    fig.update_layout(
        polar=dict(
            radialaxis=dict(visible=True, range=[0, 100], gridcolor='#30363d',
                            tickfont=dict(color='#8b949e', size=9)),
            angularaxis=dict(gridcolor='#30363d',
                             tickfont=dict(color='#c9d1d9', size=11))
        ),
        paper_bgcolor='#0d1117', plot_bgcolor='#0d1117',
        legend=dict(font=dict(color='#c9d1d9'), bgcolor='#161b22',
                    bordercolor='#30363d'),
        title=dict(text='🕸️ Skill Coverage Radar', font=dict(color='#00d4ff', size=16),
                   x=0.5),
        height=480
    )
    fig.show()


def plot_gap_priority(gaps: GapAnalysis):
    if not gaps.gap_priority:
        display(HTML('<div style="color:#2ecc71;padding:16px">🎉 No critical gaps to display!</div>'))
        return

    items   = sorted(gaps.gap_priority.items(), key=lambda x: -x[1])
    skills  = [i[0] for i in items]
    weeks   = [i[1] for i in items]
    colors  = ['#ff6b6b' if w >= 8 else '#f39c12' if w >= 4 else '#2ecc71' for w in weeks]

    fig = go.Figure(go.Bar(
        x=weeks, y=skills, orientation='h',
        marker=dict(color=colors, line=dict(color='#30363d', width=0.5)),
        text=[f'{w}w' for w in weeks], textposition='outside',
        textfont=dict(color='#c9d1d9')
    ))
    fig.update_layout(
        title=dict(text='⏱️ Skill Gaps — Estimated Learning Time (weeks)',
                   font=dict(color='#00d4ff', size=16), x=0.5),
        xaxis=dict(title='Weeks to Learn', gridcolor='#30363d',
                   tickfont=dict(color='#8b949e')),
        yaxis=dict(tickfont=dict(color='#c9d1d9'), autorange='reversed'),
        paper_bgcolor='#0d1117', plot_bgcolor='#161b22',
        height=max(300, len(skills) * 45 + 80),
        margin=dict(l=10, r=60, t=50, b=30)
    )
    fig.show()


def plot_roadmap(roadmap: List[Dict]):
    if not roadmap:
        return

    phase_colors = ['#00d4ff', '#a855f7', '#2ecc71', '#f39c12', '#ff6b6b']
    html = '<div style="background:#161b22;border:1px solid #30363d;border-radius:16px;padding:24px;margin:12px 0">'
    html += '<h3 style="color:#00d4ff;font-family:monospace;margin-top:0">🗺️ Personalized Learning Roadmap</h3>'

    for i, phase in enumerate(roadmap):
        c = phase_colors[i % len(phase_colors)]
        skills_html = ' '.join(
            f'<span style="background:{c}22;color:{c};border:1px solid {c}55;'
            f'border-radius:12px;padding:2px 9px;font-size:0.8em;margin:2px;display:inline-block">{s}</span>'
            for s in phase.get('skills', []))
        actions_html = ''.join(
            f'<li style="color:#c9d1d9;margin:3px 0">{a}</li>'
            for a in phase.get('actions', []))
        resources_html = ''.join(
            f'<span style="color:#8b949e;font-size:0.85em">📚 {r}</span><br>'
            for r in phase.get('resources', []))

        html += f'''
        <div style="border-left:3px solid {c};padding-left:20px;margin-bottom:24px;position:relative">
          <div style="position:absolute;left:-8px;top:0;width:14px;height:14px;
                      border-radius:50%;background:{c};box-shadow:0 0 10px {c}88"></div>
          <div style="display:flex;gap:12px;align-items:baseline;flex-wrap:wrap">
            <span style="color:{c};font-size:0.75em;font-weight:700;letter-spacing:1px">PHASE {phase.get('phase', i+1)}</span>
            <span style="color:#c9d1d9;font-size:1.05em;font-weight:600">{phase.get('title', '')}</span>
            <span style="background:#30363d;color:#8b949e;border-radius:8px;padding:1px 8px;
                         font-size:0.78em">{phase.get('duration', '')}</span>
          </div>
          <div style="margin:8px 0">{skills_html}</div>
          <ul style="margin:8px 0;padding-left:20px">{actions_html}</ul>
          {f'<div style="margin-top:6px">{resources_html}</div>' if resources_html else ''}
          {f"<div style=\"background:#0d1117;border:1px solid {c}44;border-radius:8px;padding:8px 12px;margin-top:8px;\"><span style=\"color:{c};font-size:0.8em;font-weight:700\">🎯 MILESTONE: </span><span style=\"color:#c9d1d9;font-size:0.85em\">{phase.get('milestone', '')}</span></div>" if phase.get('milestone') else ''}
        </div>'''

    html += '</div>'
    display(HTML(html))


def plot_trajectory(trajectory: List[Dict]):
    if not trajectory:
        return

    years   = [t.get('year', i) for i, t in enumerate(trajectory)]
    scores  = [t.get('readiness_score', 50) for t in trajectory]
    roles   = [t.get('role', '') for t in trajectory]
    salaries = [t.get('salary_range', '') for t in trajectory]

    # Parse salary midpoints for plotting
    salary_mid = []
    for s in salaries:
        nums = re.findall(r'\d+', s.replace(',', ''))
        if len(nums) >= 2:
            salary_mid.append((int(nums[-2]) + int(nums[-1])) / 2)
        elif nums:
            salary_mid.append(int(nums[0]))
        else:
            salary_mid.append(salary_mid[-1] if salary_mid else 80000)

    fig = make_subplots(
        rows=1, cols=2,
        subplot_titles=('Career Readiness Score', 'Estimated Salary Trajectory'),
        specs=[[{'type': 'scatter'}, {'type': 'bar'}]]
    )

    # Readiness line
    fig.add_trace(go.Scatter(
        x=years, y=scores, mode='lines+markers+text',
        text=[f'{s:.0f}%' for s in scores],
        textposition='top center',
        line=dict(color='#00d4ff', width=3),
        marker=dict(size=12, color='#00d4ff',
                    line=dict(color='#0d1117', width=2)),
        name='Readiness %',
        customdata=roles,
        hovertemplate='<b>Year %{x}</b><br>%{customdata}<br>Readiness: %{y:.0f}%<extra></extra>'
    ), row=1, col=1)

    # Add threshold line at 80%
    fig.add_hline(y=80, line_dash='dash', line_color='#2ecc71',
                  annotation_text='Ready threshold', row=1, col=1)

    # Salary bars
    bar_colors = ['#00d4ff' if i == 0 else '#a855f7' for i in range(len(years))]
    fig.add_trace(go.Bar(
        x=[f'Yr {y}' for y in years], y=salary_mid,
        marker=dict(color=bar_colors, opacity=0.85),
        text=[f'${v/1000:.0f}k' for v in salary_mid],
        textposition='outside', textfont=dict(color='#c9d1d9'),
        name='Salary ($USD)',
        customdata=roles,
        hovertemplate='<b>%{customdata}</b><br>~$%{y:,.0f}/yr<extra></extra>'
    ), row=1, col=2)

    fig.update_layout(
        paper_bgcolor='#0d1117', plot_bgcolor='#161b22',
        font=dict(color='#c9d1d9'),
        title=dict(text='📈 5-Year Career Trajectory Prediction',
                   font=dict(color='#00d4ff', size=16), x=0.5),
        showlegend=False, height=420,
    )
    for axis in ['xaxis', 'yaxis', 'xaxis2', 'yaxis2']:
        fig.update_layout(**{axis: dict(gridcolor='#30363d', showgrid=True)})

    fig.show()

    # Role timeline
    html = '<div style="background:#161b22;border:1px solid #30363d;border-radius:12px;padding:20px;margin:12px 0">'
    html += '<h4 style="color:#00d4ff;font-family:monospace;margin-top:0">📅 Career Milestone Timeline</h4>'
    html += '<div style="display:flex;gap:0;overflow-x:auto;padding-bottom:8px">'

    milestone_colors = ['#00d4ff', '#a855f7', '#2ecc71', '#f39c12', '#ff6b6b', '#fb923c']
    for i, t in enumerate(trajectory):
        c = milestone_colors[i % len(milestone_colors)]
        skills_html = '<br>'.join(
            f'<span style="color:{c};font-size:0.75em">+ {s}</span>'
            for s in t.get('skills_to_add', [])[:3])
        html += f'''
        <div style="min-width:160px;border:1px solid {c}44;border-radius:10px;
                    padding:12px;margin-right:8px;background:{c}0a;">
          <div style="color:{c};font-size:0.75em;font-weight:700">{t.get('label', f'Year {i}')}</div>
          <div style="color:#c9d1d9;font-size:0.85em;font-weight:600;margin:4px 0">{t.get('role', '')[:30]}</div>
          <div style="color:#8b949e;font-size:0.75em">{t.get('salary_range', '')}</div>
          <div style="margin-top:6px">{skills_html}</div>
        </div>'''
    html += '</div></div>'
    display(HTML(html))


def display_interview_questions(qa: List[Dict]):
    type_colors = {
        'Technical': '#00d4ff', 'Behavioral': '#2ecc71',
        'System Design': '#a855f7', 'Situational': '#f39c12'
    }
    diff_colors = {'Easy': '#2ecc71', 'Medium': '#f39c12', 'Hard': '#ff6b6b'}

    html = '<div style="background:#161b22;border:1px solid #30363d;border-radius:16px;padding:24px;margin:12px 0">'
    html += '<h3 style="color:#00d4ff;font-family:monospace;margin-top:0">🎤 Interview Simulation — Question Bank</h3>'

    for i, q in enumerate(qa):
        qtype = q.get('type', 'Technical')
        diff  = q.get('difficulty', 'Medium')
        tc    = type_colors.get(qtype, '#00d4ff')
        dc    = diff_colors.get(diff, '#f39c12')

        cover = ''.join(f'<li style="color:#c9d1d9;margin:2px 0">{p}</li>'
                        for p in q.get('what_to_cover', []))
        html += f'''
        <div style="border:1px solid #30363d;border-radius:10px;padding:16px;margin-bottom:14px;
                    background:#0d1117;border-left:3px solid {tc}">
          <div style="display:flex;gap:8px;margin-bottom:10px;flex-wrap:wrap">
            <span style="background:{tc}22;color:{tc};border:1px solid {tc}55;
                         border-radius:12px;padding:2px 10px;font-size:0.78em">{qtype}</span>
            <span style="background:{dc}22;color:{dc};border:1px solid {dc}55;
                         border-radius:12px;padding:2px 10px;font-size:0.78em">{diff}</span>
            <span style="color:#8b949e;font-size:0.8em;margin-left:auto">Q{i+1}</span>
          </div>
          <div style="color:#e6edf3;font-size:1em;font-weight:600;margin-bottom:10px">{q.get('question', '')}</div>
          {f'<div style="background:#161b22;border-radius:8px;padding:10px;margin-top:8px"><div style="color:#8b949e;font-size:0.78em;font-weight:700;margin-bottom:4px">KEY POINTS TO COVER</div><ul style="margin:0;padding-left:18px">{cover}</ul></div>' if cover else ''}
          {f'<div style="margin-top:8px;padding:8px 12px;background:#1e2a1e;border-radius:8px"><span style="color:#2ecc71;font-size:0.78em;font-weight:700">💡 STRONG ANSWER: </span><span style="color:#c9d1d9;font-size:0.85em">{q.get("sample_strong_answer", "")}</span></div>' if q.get('sample_strong_answer') else ''}
          {f'<div style="margin-top:6px;padding:6px 12px;background:#2a1e1e;border-radius:8px"><span style="color:#ff6b6b;font-size:0.78em;font-weight:700">⚠️ RED FLAG: </span><span style="color:#8b949e;font-size:0.82em">{q.get("red_flags", "")}</span></div>' if q.get('red_flags') else ''}
        </div>'''

    html += '</div>'
    display(HTML(html))


print('✅ Visualization engine ready')
print('   Functions: display_header | display_score_card | plot_skill_radar')
print('              plot_gap_priority | plot_roadmap | plot_trajectory')
print('              display_interview_questions')

## 📄 Cell 6 — Input: Upload Resume & Job Description

In [ ]:
# ─── Resume & JD Input System ─────────────────────────────────────────────────
# You can EITHER upload files OR paste text directly below.
# If you paste text, uploads are skipped.

# ══════════════════════════════════════════════════════════
#   OPTION A: PASTE YOUR TEXT DIRECTLY HERE
#   (Leave empty to use file upload instead)
# ══════════════════════════════════════════════════════════

PASTE_RESUME = """  
"""  # ← Paste your resume text between the triple quotes

PASTE_JD = """
"""  # ← Paste the job description text between the triple quotes


# ─── Demo Data (used if both paste fields are empty) ──────────────────────────
DEMO_RESUME = """
ALEX CHEN — Machine Learning Engineer
Email: alex.chen@email.com | LinkedIn: linkedin.com/in/alexchen | GitHub: github.com/alexchen

SUMMARY
Machine Learning Engineer with 4 years of experience building production ML systems.
Strong background in Python, deep learning, and deploying models at scale.
Passionate about solving real-world problems with data-driven solutions.

EXPERIENCE
Senior ML Engineer — DataFlow Inc (2022–Present)
• Built and deployed recommendation system using PyTorch, serving 2M+ users
• Designed ETL pipelines with Apache Airflow and PostgreSQL
• Reduced model inference latency by 40% using model quantization
• Collaborated with cross-functional teams using Agile/Scrum methodology

ML Engineer — StartupAI (2020–2022)
• Developed NLP models for sentiment analysis using Python, TensorFlow, scikit-learn
• Built REST API endpoints with Flask for model serving
• Implemented automated testing and CI/CD with GitHub Actions
• Managed datasets and experiments using pandas and numpy

EDUCATION
M.S. Computer Science — Stanford University (2020)
B.S. Mathematics — UC Berkeley (2018)

SKILLS
Languages: Python, SQL, Bash, R
ML/AI: PyTorch, TensorFlow, scikit-learn, pandas, numpy, Keras, NLP, Computer Vision
Cloud: AWS (EC2, S3, Lambda, SageMaker), Docker
Databases: PostgreSQL, MongoDB, Redis
Tools: Git, Airflow, Flask, GitHub Actions, Jupyter, Linux

PROJECTS
• Real-time fraud detection system (Python, Kafka, Redis, PostgreSQL)
• Open-source NLP toolkit with 800+ GitHub stars
• LLM-powered document summarizer (LangChain, OpenAI API)

CERTIFICATIONS
• AWS Certified Machine Learning – Specialty
• Deep Learning Specialization — Coursera (Andrew Ng)
"""

DEMO_JD = """
Senior ML Engineer — TechUnicorn Inc

About the Role:
We're looking for a Senior ML Engineer to join our AI Platform team and help scale our
machine learning infrastructure to support 10M+ users.

Requirements (Must-Have):
• 5+ years experience in machine learning engineering
• Expert-level Python and PyTorch
• Strong experience with Kubernetes and MLOps practices
• Apache Spark for large-scale data processing
• MLflow or similar experiment tracking
• Strong system design skills for distributed ML systems
• Experience with LLMs and RAG architectures

Nice to Have:
• Terraform for infrastructure as code
• Rust for performance-critical components
• GraphQL API design
• Fine-tuning large language models
• Contribution to open-source ML projects

What You'll Do:
• Design and deploy ML infrastructure at scale
• Build ML pipelines that process petabytes of data
• Lead technical architecture decisions for the AI platform
• Mentor junior engineers and drive engineering excellence
• Collaborate with research teams to productionize cutting-edge models

Compensation: $180,000 - $240,000 + equity + benefits
Location: Remote-first (US timezone preferred)
"""


# ─── Load text logic ──────────────────────────────────────────────────────────
resume_text = PASTE_RESUME.strip()
jd_text     = PASTE_JD.strip()

if not resume_text or not jd_text:
    if IN_COLAB:
        display(HTML('''
        <div style="background:#161b22;border:1px solid #30363d;border-radius:12px;
                    padding:20px;color:#c9d1d9;">
          <h3 style="color:#00d4ff;margin-top:0">📤 File Upload Mode</h3>
          <p>You can upload your resume and job description files below.<br>
             <b>Supported formats:</b> .pdf, .docx, .txt<br>
             Or just continue — demo data will be used.</p>
        </div>
        '''))

        print('Upload your RESUME file (or press Cancel to use demo):')
        try:
            uploaded = colab_files.upload()
            if uploaded:
                fname = list(uploaded.keys())[0]
                resume_text = DocumentParser.parse(fname, uploaded[fname])
                print(f'  ✅ Resume loaded: {fname} ({len(resume_text)} chars)')
        except Exception as e:
            print(f'  ℹ️  Upload skipped or failed: {e}')

        if not resume_text:
            print('  ℹ️  Using demo resume')
            resume_text = DEMO_RESUME

        print('\nUpload the JOB DESCRIPTION file (or press Cancel to use demo):')
        try:
            uploaded = colab_files.upload()
            if uploaded:
                fname = list(uploaded.keys())[0]
                jd_text = DocumentParser.parse(fname, uploaded[fname])
                print(f'  ✅ JD loaded: {fname} ({len(jd_text)} chars)')
        except Exception as e:
            print(f'  ℹ️  Upload skipped or failed: {e}')

        if not jd_text:
            print('  ℹ️  Using demo job description')
            jd_text = DEMO_JD
    else:
        print('ℹ️  No text pasted. Using demo data.')
        resume_text = resume_text or DEMO_RESUME
        jd_text     = jd_text or DEMO_JD

twin.resume_text = resume_text
twin.jd_text     = jd_text

print(f'\n✅ Resume loaded  : {len(resume_text):,} characters')
print(f'✅ Job desc loaded : {len(jd_text):,} characters')
print('\nReady for analysis. Run Cell 7 to start! ▶️')

## 🧠 Cell 7 — Run Full AI Analysis (All 6 Engines)

In [ ]:
# ─── Full Pipeline Execution ──────────────────────────────────────────────────

def run_progress(step: int, total: int, label: str):
    pct  = int(step / total * 100)
    bar  = '█' * (pct // 5) + '░' * (20 - pct // 5)
    print(f'  [{bar}] {pct:3d}%  {label}')


display(HTML('''
<div style="background:linear-gradient(135deg,#0d1117,#1a1a2e);border:1px solid #00d4ff44;
            border-radius:12px;padding:20px;margin:12px 0">
  <h2 style="color:#00d4ff;font-family:monospace;margin:0">🔄 Running AI Career Analysis...</h2>
  <p style="color:#8b949e;margin:8px 0 0">6 engines · ~60-90 seconds · powered by Gemini 1.5 Flash</p>
</div>
'''))

t0 = time.time()

# Step 1: Local skill extraction
print('\n📊 Step 1/6  —  Local Skill Extraction')
resume_local = extract_skills_local(twin.resume_text)
jd_local     = extract_skills_local(twin.jd_text)
res_exp      = extract_experience_years(twin.resume_text)
total_local  = sum(len(v) for v in resume_local.values())
run_progress(1, 6, f'Found {total_local} skills across {len(resume_local)} categories')

# Step 2: Deep resume analysis
print('\n🤖 Step 2/6  —  AI Resume Analysis')
twin.skill_profile = ai_engine.analyze_resume(twin.resume_text, resume_local)
if res_exp > 0 and twin.skill_profile.experience_years == 0:
    twin.skill_profile.experience_years = res_exp
run_progress(2, 6, f'{len(twin.skill_profile.all_skills)} skills · {twin.skill_profile.experience_years}yr exp · {twin.skill_profile.current_role}')

# Step 3: JD parsing
print('\n📋 Step 3/6  —  Job Description Intelligence')
twin.job_profile = ai_engine.parse_job_description(twin.jd_text, jd_local)
run_progress(3, 6, f'{twin.job_profile.title} · {twin.job_profile.seniority} · {len(twin.job_profile.required_skills)} requirements')

# Step 4: Gap analysis
print('\n🎯 Step 4/6  —  Skill Gap Detection')
twin.gap_analysis = ai_engine.analyze_gaps(twin.skill_profile, twin.job_profile)
run_progress(4, 6, f'{twin.gap_analysis.match_score}% match · {len(twin.gap_analysis.critical_gaps)} gaps · {twin.gap_analysis.readiness_level}')

# Step 5: Roadmap generation
print('\n🗺️  Step 5/6  —  Learning Roadmap Generation')
twin.roadmap = ai_engine.generate_roadmap(twin.skill_profile, twin.job_profile, twin.gap_analysis)
run_progress(5, 6, f'{len(twin.roadmap)} phases generated')

# Step 6: Interview questions + trajectory (parallel prompt effect)
print('\n🎤 Step 6/6  —  Interview Sim + Career Trajectory')
twin.interview_qa  = ai_engine.generate_interview_questions(twin.skill_profile, twin.job_profile, twin.gap_analysis)
twin.trajectory    = ai_engine.predict_trajectory(twin.skill_profile, twin.job_profile, twin.gap_analysis)
run_progress(6, 6, f'{len(twin.interview_qa)} questions · {len(twin.trajectory)} year trajectory')

elapsed = time.time() - t0
print(f'\n✅ Analysis complete in {elapsed:.1f}s')
print(f'   Resume : {twin.skill_profile.current_role}  ({twin.skill_profile.experience_years}yr)')
print(f'   Target : {twin.job_profile.title} @ {twin.job_profile.company}')
print(f'   Score  : {twin.gap_analysis.match_score}%  {twin.gap_analysis.readiness_level}')
print(f'   Gaps   : {len(twin.gap_analysis.critical_gaps)} critical · {len(twin.gap_analysis.nice_to_have_gaps)} nice-to-have')
print('\n▶️  Run Cell 8 to see full results dashboard!')

## 📊 Cell 8 — Full Results Dashboard

In [ ]:
# ─── Results Dashboard ────────────────────────────────────────────────────────

if not twin.skill_profile:
    raise RuntimeError('❌ Run Cell 7 first!')

# ── Header ───────────────────────────────────────────────────────────────────
display_header(
    '🚀 AI CAREER TWIN — RESULTS',
    f'{twin.skill_profile.current_role}  →  {twin.job_profile.title} @ {twin.job_profile.company}'
)

# ── Section 1: Profile Cards ──────────────────────────────────────────────────
display(HTML(f'''
<div style="display:flex;gap:12px;flex-wrap:wrap;margin:12px 0">
  <div style="flex:1;min-width:220px;background:#161b22;border:1px solid #30363d;
              border-radius:12px;padding:18px;border-top:3px solid #00d4ff">
    <div style="color:#00d4ff;font-size:0.75em;font-weight:700;letter-spacing:1px">👤 CANDIDATE</div>
    <div style="color:#e6edf3;font-size:1.1em;font-weight:700;margin:6px 0">{twin.skill_profile.current_role}</div>
    <div style="color:#8b949e;font-size:0.85em">{twin.skill_profile.experience_years}yr exp · {twin.skill_profile.education}</div>
    <div style="color:#8b949e;font-size:0.85em;margin-top:4px">{len(twin.skill_profile.all_skills)} skills detected</div>
  </div>
  <div style="flex:1;min-width:220px;background:#161b22;border:1px solid #30363d;
              border-radius:12px;padding:18px;border-top:3px solid #ff6b6b">
    <div style="color:#ff6b6b;font-size:0.75em;font-weight:700;letter-spacing:1px">🎯 TARGET ROLE</div>
    <div style="color:#e6edf3;font-size:1.1em;font-weight:700;margin:6px 0">{twin.job_profile.title}</div>
    <div style="color:#8b949e;font-size:0.85em">{twin.job_profile.company} · {twin.job_profile.seniority}</div>
    <div style="color:#8b949e;font-size:0.85em;margin-top:4px">{len(twin.job_profile.required_skills)} requirements</div>
  </div>
  <div style="flex:1;min-width:220px;background:#161b22;border:1px solid #30363d;
              border-radius:12px;padding:18px;border-top:3px solid #2ecc71">
    <div style="color:#2ecc71;font-size:0.75em;font-weight:700;letter-spacing:1px">📈 ANALYSIS</div>
    <div style="color:#e6edf3;font-size:1.1em;font-weight:700;margin:6px 0">{twin.gap_analysis.match_score}% Match</div>
    <div style="color:#8b949e;font-size:0.85em">{twin.gap_analysis.readiness_level}</div>
    <div style="color:#8b949e;font-size:0.85em;margin-top:4px">{len(twin.gap_analysis.critical_gaps)} gaps to close</div>
  </div>
</div>
'''))

# ── Section 2: Score Card ─────────────────────────────────────────────────────
display(HTML('<h3 style="color:#00d4ff;font-family:monospace">⚡ Match Score & Skill Gap Analysis</h3>'))
display_score_card(
    twin.gap_analysis.match_score,
    twin.gap_analysis.readiness_level,
    twin.gap_analysis.matching_skills,
    twin.gap_analysis.critical_gaps
)

# ── Section 3: Radar Chart ────────────────────────────────────────────────────
display(HTML('<h3 style="color:#00d4ff;font-family:monospace">🕸️ Skill Coverage Radar</h3>'))
plot_skill_radar(twin.skill_profile, twin.job_profile, twin.gap_analysis)

# ── Section 4: Gap Priority ───────────────────────────────────────────────────
display(HTML('<h3 style="color:#00d4ff;font-family:monospace">⏱️ Skill Gap Priority Chart</h3>'))
plot_gap_priority(twin.gap_analysis)

print('✅ Score card, radar and gap charts rendered.')
print('▶️  Scroll down to view full dashboard!')

## 🗺️ Cell 9 — Learning Roadmap & Career Trajectory

In [ ]:
if not twin.roadmap:
    raise RuntimeError('❌ Run Cell 7 first!')

# Roadmap
display(HTML('<h3 style="color:#00d4ff;font-family:monospace">🗺️ Personalized Learning Roadmap</h3>'))
plot_roadmap(twin.roadmap)

# Total timeline summary
total_weeks = 0
for phase in twin.roadmap:
    m = re.search(r'(\d+)', phase.get('duration', '0'))
    if m:
        total_weeks += int(m.group(1))

display(HTML(f'''
<div style="background:#1e3a1e;border:1px solid #2ecc71;border-radius:10px;
            padding:16px;margin:12px 0">
  <span style="color:#2ecc71;font-weight:700">⏱️ Total estimated prep time: 
  {total_weeks} weeks ({total_weeks//4} months)</span><br>
  <span style="color:#8b949e;font-size:0.88em">Assuming 10-15 hours/week of focused study</span>
</div>
'''))

# Trajectory
display(HTML('<h3 style="color:#00d4ff;font-family:monospace">📈 5-Year Career Trajectory Prediction</h3>'))
plot_trajectory(twin.trajectory)

print('✅ Roadmap and trajectory rendered!')

## 🎤 Cell 10 — Interview Simulation

In [ ]:
if not twin.interview_qa:
    raise RuntimeError('❌ Run Cell 7 first!')

display_header(
    '🎤 INTERVIEW SIMULATION',
    f'Tailored questions for {twin.job_profile.title} · {twin.job_profile.seniority} level'
)

# Stats bar
type_counts = {}
diff_counts = {}
for q in twin.interview_qa:
    t = q.get('type', 'Technical')
    d = q.get('difficulty', 'Medium')
    type_counts[t] = type_counts.get(t, 0) + 1
    diff_counts[d] = diff_counts.get(d, 0) + 1

tc_html = ' · '.join(
    f'<span style="color:#00d4ff">{v}× {k}</span>' for k, v in type_counts.items())
dc_html = ' · '.join(
    f'<span style="color:{"#2ecc71" if k=="Easy" else "#f39c12" if k=="Medium" else "#ff6b6b"}">{v}× {k}</span>'
    for k, v in diff_counts.items())

display(HTML(f'''
<div style="background:#161b22;border:1px solid #30363d;border-radius:10px;
            padding:12px 16px;margin:12px 0;font-size:0.9em">
  <span style="color:#8b949e">Types: </span>{tc_html}
  &nbsp;&nbsp;|&nbsp;&nbsp;
  <span style="color:#8b949e">Difficulty: </span>{dc_html}
  &nbsp;&nbsp;|&nbsp;&nbsp;
  <span style="color:#8b949e">Total: </span><span style="color:#c9d1d9">{len(twin.interview_qa)} questions</span>
</div>
'''))

display_interview_questions(twin.interview_qa)

# Interview tips
display(HTML('''
<div style="background:#161b22;border:1px solid #30363d;border-radius:12px;padding:20px;margin:12px 0">
  <h4 style="color:#f39c12;font-family:monospace;margin-top:0">💡 Interview Strategy Tips</h4>
  <div style="color:#c9d1d9;font-size:0.9em;line-height:1.8">
    <b style="color:#00d4ff">STAR Method</b> — For behavioral questions: Situation → Task → Action → Result<br>
    <b style="color:#00d4ff">Think aloud</b> — For system design: walk through your reasoning before diving in<br>
    <b style="color:#00d4ff">Numbers matter</b> — Always quantify impact ("reduced latency by 40%", "served 2M users")<br>
    <b style="color:#00d4ff">Address gaps honestly</b> — "I haven't used X in production, but I've done Y which is similar"<br>
    <b style="color:#00d4ff">Ask clarifying questions</b> — Especially for ambiguous system design prompts
  </div>
</div>
'''))

print('✅ Interview simulation complete!')

## 💾 Cell 11 — Export Full Report

In [ ]:
# ─── Export Full HTML Report ──────────────────────────────────────────────────

if not twin.skill_profile:
    raise RuntimeError('❌ Run Cell 7 first!')

def generate_html_report(twin: CareerTwin) -> str:
    g   = twin.gap_analysis
    sp  = twin.skill_profile
    jp  = twin.job_profile

    match_color = ('#2ecc71' if g.match_score >= 80 else
                   '#f39c12' if g.match_score >= 60 else
                   '#e67e22' if g.match_score >= 40 else '#e74c3c')

    match_pills = ''.join(
        f'<span class="pill match">{s}</span>' for s in g.matching_skills)
    gap_pills   = ''.join(
        f'<span class="pill gap">{s}</span>' for s in g.critical_gaps)

    roadmap_html = ''
    for phase in twin.roadmap:
        skills_str = ', '.join(phase.get('skills', []))
        actions_li = ''.join(f'<li>{a}</li>' for a in phase.get('actions', []))
        roadmap_html += f'''
        <div class="phase">
          <div class="phase-header">Phase {phase.get("phase","")} · {phase.get("title","")} · {phase.get("duration","")}</div>
          <div class="phase-skills">Skills: {skills_str}</div>
          <ul>{actions_li}</ul>
          {f'<div class="milestone">🎯 {phase.get("milestone","")}</div>' if phase.get('milestone') else ''}
        </div>'''

    iq_html = ''
    for i, q in enumerate(twin.interview_qa):
        iq_html += f'''
        <div class="question">
          <div class="q-meta">{q.get("type","")} · {q.get("difficulty","")}</div>
          <div class="q-text">Q{i+1}: {q.get("question","")}</div>
          {f'<div class="q-answer">💡 {q.get("sample_strong_answer","")}</div>' if q.get('sample_strong_answer') else ''}
        </div>'''

    traj_rows = ''.join(
        f'<tr><td>{t.get("label","")}</td><td>{t.get("role","")}</td>'
        f'<td>{t.get("salary_range","")}</td>'
        f'<td>{t.get("readiness_score",0):.0f}%</td></tr>'
        for t in twin.trajectory)

    return f'''<!DOCTYPE html>
<html lang="en">
<head>
<meta charset="UTF-8">
<meta name="viewport" content="width=device-width,initial-scale=1">
<title>AI Career Twin Report</title>
<style>
  *{{box-sizing:border-box;margin:0;padding:0}}
  body{{background:#0d1117;color:#c9d1d9;font-family:'Segoe UI',system-ui,sans-serif;padding:24px;max-width:960px;margin:0 auto}}
  h1{{color:#00d4ff;font-family:monospace;font-size:2em;margin-bottom:4px}}
  h2{{color:#00d4ff;font-family:monospace;margin:28px 0 12px;border-bottom:1px solid #30363d;padding-bottom:6px}}
  .subtitle{{color:#8b949e;margin-bottom:24px}}
  .card{{background:#161b22;border:1px solid #30363d;border-radius:12px;padding:20px;margin-bottom:16px}}
  .cards{{display:flex;gap:12px;flex-wrap:wrap}}
  .cards .card{{flex:1;min-width:200px}}
  .label{{color:#00d4ff;font-size:0.75em;font-weight:700;letter-spacing:1px;margin-bottom:6px}}
  .value{{color:#e6edf3;font-size:1.1em;font-weight:700}}
  .sub{{color:#8b949e;font-size:0.85em;margin-top:4px}}
  .score-circle{{width:100px;height:100px;border-radius:50%;border:5px solid {match_color};
    display:flex;align-items:center;justify-content:center;margin:0 auto 12px;
    box-shadow:0 0 20px {match_color}44}}
  .score-num{{color:{match_color};font-size:1.8em;font-weight:900;text-align:center}}
  .pill{{display:inline-block;border-radius:20px;padding:3px 10px;font-size:0.8em;margin:2px}}
  .match{{background:#1e3a2e;color:#2ecc71;border:1px solid #2ecc71}}
  .gap{{background:#3a1e1e;color:#ff6b6b;border:1px solid #ff6b6b}}
  .phase{{border-left:3px solid #00d4ff;padding:12px 16px;margin-bottom:16px;
    background:#0d1117;border-radius:0 8px 8px 0}}
  .phase-header{{color:#00d4ff;font-weight:700;margin-bottom:6px}}
  .phase-skills{{color:#8b949e;font-size:0.85em;margin-bottom:6px}}
  .phase ul{{padding-left:18px;color:#c9d1d9}}
  .phase li{{margin:3px 0}}
  .milestone{{background:#1a1a2e;padding:8px;border-radius:6px;margin-top:8px;
    color:#a855f7;font-size:0.88em}}
  .question{{border:1px solid #30363d;border-radius:8px;padding:14px;margin-bottom:12px;
    border-left:3px solid #00d4ff}}
  .q-meta{{color:#00d4ff;font-size:0.78em;margin-bottom:6px}}
  .q-text{{color:#e6edf3;font-weight:600;margin-bottom:8px}}
  .q-answer{{background:#1e2a1e;padding:8px;border-radius:6px;color:#c9d1d9;font-size:0.88em}}
  table{{width:100%;border-collapse:collapse;margin-top:12px}}
  th{{background:#1e2a3a;color:#00d4ff;padding:10px;text-align:left;font-size:0.85em}}
  td{{padding:10px;border-bottom:1px solid #21262d;color:#c9d1d9;font-size:0.9em}}
  .footer{{text-align:center;color:#30363d;margin-top:40px;font-size:0.8em}}
</style>
</head>
<body>
<h1>🚀 AI Career Twin Report</h1>
<p class="subtitle">Generated {datetime.now().strftime('%B %d, %Y at %H:%M')} · Powered by Gemini AI</p>

<div class="cards">
  <div class="card"><div class="label">👤 CANDIDATE</div>
    <div class="value">{sp.current_role}</div>
    <div class="sub">{sp.experience_years}yr exp · {sp.education}</div></div>
  <div class="card"><div class="label">🎯 TARGET</div>
    <div class="value">{jp.title}</div>
    <div class="sub">{jp.company} · {jp.seniority}</div></div>
  <div class="card" style="text-align:center">
    <div class="score-circle"><div class="score-num">{g.match_score:.0f}%</div></div>
    <div style="color:{match_color};font-weight:600">{g.readiness_level}</div></div>
</div>

<h2>⚡ Skill Analysis</h2>
<div class="card">
  <div class="label" style="color:#2ecc71">MATCHING SKILLS ({len(g.matching_skills)})</div>
  <div style="margin:8px 0">{match_pills if match_pills else '<span style="color:#666">None</span>'}</div>
  <div class="label" style="color:#ff6b6b;margin-top:12px">CRITICAL GAPS ({len(g.critical_gaps)})</div>
  <div style="margin:8px 0">{gap_pills if gap_pills else '<span style="color:#2ecc71">No gaps — ready to apply!</span>'}</div>
</div>

<h2>🗺️ Learning Roadmap</h2>
{roadmap_html}

<h2>🎤 Interview Questions</h2>
{iq_html}

<h2>📈 Career Trajectory</h2>
<table><tr><th>Timeline</th><th>Role</th><th>Salary Range</th><th>Readiness</th></tr>
{traj_rows}</table>

<p class="footer">AI Career Twin · Built with Google Gemini · {datetime.now().year}</p>
</body></html>'''


print('⏳ Generating HTML report...')
html_report = generate_html_report(twin)
report_filename = f'career_twin_report_{datetime.now().strftime("%Y%m%d_%H%M%S")}.html'

with open(report_filename, 'w', encoding='utf-8') as f:
    f.write(html_report)

print(f'✅ Report saved: {report_filename} ({len(html_report):,} chars)')

# Also export JSON summary
json_report = {
    'generated_at': datetime.now().isoformat(),
    'candidate': {
        'role': twin.skill_profile.current_role,
        'experience_years': twin.skill_profile.experience_years,
        'education': twin.skill_profile.education,
        'skills': twin.skill_profile.all_skills,
        'certifications': twin.skill_profile.certifications,
    },
    'target_job': {
        'title': twin.job_profile.title,
        'company': twin.job_profile.company,
        'seniority': twin.job_profile.seniority,
        'required_skills': twin.job_profile.required_skills,
    },
    'gap_analysis': {
        'match_score': twin.gap_analysis.match_score,
        'readiness_level': twin.gap_analysis.readiness_level,
        'matching_skills': twin.gap_analysis.matching_skills,
        'critical_gaps': twin.gap_analysis.critical_gaps,
        'nice_to_have_gaps': twin.gap_analysis.nice_to_have_gaps,
        'learning_time_weeks': twin.gap_analysis.gap_priority,
    },
    'roadmap': twin.roadmap,
    'trajectory': twin.trajectory,
    'interview_questions': twin.interview_qa,
}

json_filename = report_filename.replace('.html', '.json')
with open(json_filename, 'w') as f:
    json.dump(json_report, f, indent=2)

print(f'✅ JSON data saved: {json_filename}')

if IN_COLAB:
    print('\n⬇️  Downloading files...')
    colab_files.download(report_filename)
    colab_files.download(json_filename)

display(HTML(f'''
<div style="background:#1e3a1e;border:1px solid #2ecc71;border-radius:12px;
            padding:20px;margin:16px 0;">
  <h3 style="color:#2ecc71;margin:0 0 10px">✅ Export Complete!</h3>
  <div style="color:#c9d1d9">
    📄 <b style="color:#00d4ff">{report_filename}</b> — Full HTML dashboard (open in browser)<br>
    📊 <b style="color:#00d4ff">{json_filename}</b> — Raw JSON data for integrations
  </div>
</div>
'''))

---
## 🏁 You're Done!

```
  ┌─────────────────────────────────────────────────────────┐
  │  WHAT YOUR AI CAREER TWIN JUST DID                      │
  │                                                         │
  │  ✅  Parsed your resume with AI + local NLP             │
  │  ✅  Analyzed the job description requirements          │
  │  ✅  Detected skill gaps with priority scores           │
  │  ✅  Generated a phase-based learning roadmap           │
  │  ✅  Created tailored interview questions               │
  │  ✅  Predicted your 5-year career trajectory            │
  │  ✅  Exported full HTML + JSON report                   │
  └─────────────────────────────────────────────────────────┘
```

### 🔄 To analyze a different job — just:
1. Update `PASTE_RESUME` and `PASTE_JD` in **Cell 6**
2. Re-run **Cell 6 → Cell 7 → Cell 8 → Cell 9 → Cell 10**

---
*Built with ❤️ using Google Gemini 1.5 Flash · dlib · Plotly · Python*